<a href="https://colab.research.google.com/github/kgpark88/energy-bigdata-analysis/blob/main/energy_data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 시계열 데이터 분석
- Source : https://towardsdatascience.com/8-visualizations-with-python-to-handle-multiple-time-series-data-19b5b2e66dd0
- 시계열 그래프는 긴 시퀀스로 데이터를 표현하는 데 유용한 그래프입니다. 
- 타임라인을 나타내는 X축과 값을 나타내는 Y축으로 구성되어 있습니다. 
- 시계열 그래프는 추세 및 계절 효과와 같은 통찰력 정보를 추출하는 데 도움이 될 수 있습니다.

### kaggle 패키지 설치

In [ ]:
!pip install kaggle

### kaggle API Key 업로드


In [ ]:
from google.colab import files

files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

### 라이브러리 임포트

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

### 데이터셋 다운로드
- https://www.kaggle.com/datasets/bappekim/air-pollution-in-seoul
- 2017년부터 2019년까지의 서울시 25개 구 대기오염 정보(SO2, NO2, CO, O3, PM10, PM2.5)
- PM2.5 : 직경이 2.5μm보다 작은 미세 입자

In [ ]:
!kaggle datasets download -d bappekim/air-pollution-in-seoul

In [ ]:
!ls -lt

In [ ]:
!unzip air-pollution-in-seoul.zip

In [ ]:
!ls -lt AirPollutionSeoul

Pandas로 Measurement_summary.csv 데이터를 읽기

In [ ]:
df = pd.read_csv('./AirPollutionSeoul/Measurement_summary.csv')
df.head()

In [ ]:
df.info()

### 데이터 전처리

District 컬럼 생성

In [ ]:
df['Station code'].nunique()

In [ ]:
list_scode = list(set(df['Station code']))
print(list_scode)

In [ ]:
# Address : 19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republic of Korea
list_add = list(df['Address'])
District = [i.split(', ')[2] for i in list_add]
df['District'] = District

In [ ]:
list_district = list(set(District))

일부 그래프에 적용하기 위해 YM(Year-Month), Year, Month 컬럼을 생성열을 생성  
쉽게 시각화할 수 있도록 평균 월별 DataFrame으로 그룹화

In [ ]:
# Measurement date : 2017-01-01 00:00
list_YM = [i.split(" ")[0][:-3] for i in  list(df['Measurement date'])]

In [ ]:
list_YM = [i.split(" ")[0][:-3] for i in  list(df['Measurement date'])]
list_Year = [i.split(" ")[0][0:4] for i in  list(df['Measurement date'])]
list_Month = [i.split(" ")[0][5:7] for i in  list(df['Measurement date'])]

df['YM'] = list_YM
df['Year'] = list_Year
df['Month'] = list_Month

#create a monthly dataframe
df_monthly = df.groupby(['Station code', 'District', 'YM', 'Year', 'Month']).mean()
df_monthly = df_monthly[['SO2', 'NO2', 'O3', 'CO', 'PM10', 'PM2.5']].reset_index()

df_monthly.head()

###  시계열 데이터 플로팅의 문제점 : Overlapping Plots

In [ ]:
sns.set_style('darkgrid')
sns.set(rc={'figure.figsize':(14,8)})

ax = sns.lineplot(data=df_monthly, x ='YM', y = 'PM2.5',
                  hue='District', palette='viridis',
                  legend='full', lw=3)

ax.xaxis.set_major_locator(ticker.MultipleLocator(4))
plt.legend(bbox_to_anchor=(1, 1))
plt.ylabel('PM2.5 (µg/m3)')
plt.xlabel('Year-Month')
plt.show()

- 겹치는 Graph Line은 읽기 어렵습니다. 
- 2017년에는 많은 관측소의 PM2.5 양이 같은 방향으로 갔음을 알 수 있습니다. 
- 하지만 2018년과 2019년에는  Graph Line이 임의적으로 보이고 구분이 어렵습니다.

### 시각화 아이디어


#### 1. Interactive Chart
- Plotly는 Interactive Chart를 만들기 위한 그래프 라이브러리입니다.  
- Interactive Chart는 겹치는 선이 있는 영역을 확대하는 데 도움이 됩니다.

In [ ]:
import plotly.graph_objects as go

#extract color palette, the palette can be changed
pal = list(sns.color_palette(palette='viridis', n_colors=len(list_scode)).as_hex())

fig = go.Figure()
for d,p in zip(list_district, pal):
    fig.add_trace(go.Scatter(x = df_monthly[df_monthly['District']==d]['YM'],
                             y = df_monthly[df_monthly['District']==d]['PM2.5'],
                             name = d,
                             line_color = p, 
                             fill=None))   #tozeroy 

fig.show()

 Interactive Area Chart

In [ ]:
fig = go.Figure()
for d,p in zip(list_district, pal):
    fig.add_trace(go.Scatter(x = df_monthly[df_monthly['District']==d]['YM'],
                             y = df_monthly[df_monthly['District']==d]['PM2.5'],
                             name = d,
                             line_color = p, 
                             fill='tozeroy'))   #tozeroy 

fig.show()

#### 2. Small Multiple Time Series와 하나씩 비교.
- Seaborn 라이브러리를 사용하면 작은 다중 시계열을 수행할 수 있습니다. 
- 이 플롯의 이면에 있는 아이디어는 각 라인을 다른 라인의 실루엣과 비교하면서 하나씩 플롯하는 것입니다.

In [ ]:
g = sns.relplot(data = df_monthly, x = "YM", y = "PM2.5",
                col = "District", hue = "District",
                kind = "line", palette = "Spectral",   
                linewidth = 4, zorder = 5,
                col_wrap = 5, height = 3, aspect = 1.5, legend = False
               )

#add text and silhouettes
for time, ax in g.axes_dict.items():
    ax.text(.1, .85, time,
            transform = ax.transAxes, fontweight="bold"
           )
    sns.lineplot(data = df_monthly, x = "YM", y = "PM2.5", units="District",
                 estimator = None, color= ".7", linewidth=1, ax=ax
                )

ax.set_xticks('')
g.set_titles("")
g.set_axis_labels("", "PM2.5")
g.tight_layout()

#### 3. Facet Grid로 관점(point of view) 변경
- Seaborn의 FacetGrid를 사용하여 다중 플롯 그리드를 만들 수 있습니다. 
- 이 경우 '월' 및 '연도' 속성은 각각 행과 열로 설정됩니다. 
- 다른 관점에서 값은 수직으로 월간, 수평으로 연간 비교가 동시에 가능합니다.

In [ ]:
g = sns.FacetGrid(df_monthly, col="Year", row="Month", height=4.2, aspect=1.9)
g = g.map(sns.barplot, 'District', 'PM2.5', palette='viridis', ci=None, order = list_district)

g.set_xticklabels(rotation = 90)
plt.show()

#### 4. Heat Map 사용
- 히트맵은 데이터를 색상으로 값을 표시하는 2차원 차트로 나타냅니다. 
- 시계열 데이터를 처리하기 위해 세로로 그룹을 설정하고 가로로 타임라인을 설정할 수 있습니다. 
- 색상의 차이는 그룹을 구별하는 데 도움이 됩니다.

Pivot the DataFrame

In [ ]:
df_pivot = pd.pivot_table(df_monthly,
                          values='PM2.5',
                          index='District',
                          columns='YM')
df_pivot

In [ ]:
plt.figure(figsize = (40,19))
plt.title('Average Monthly PM2.5 (mircrogram/m3)')

sns.heatmap(df_pivot, annot=True, cmap='RdYlBu_r', fmt= '.4g',)
plt.xlabel('Year-Month')
plt.ylabel('District')
plt.show()

#### 5. Radar chart 각도 사용하기
- 인터랙티브 Radar Chart를 생성하기 위해 Plotly의 산점도에 각도 축을 설정할 수 있습니다. 
- 매월은 원의 변수로 선택됩니다. 예로 2019년 25개 지역의 평균 월간 PM2.5를 비교하는 레이더 차트를 생성합니다.

In [ ]:
df_19 = df_monthly[df_monthly['Year']=='2019']

In [ ]:
import plotly.graph_objects as go

#extract color palette, the palette name can be changed
pal = list(sns.color_palette(palette='viridis', n_colors=len(list_scode)).as_hex())

months = list(reversed([str(i) for i in list(range(1,13))])) + ['12']
list_PM = [[list(df_19[df_19['District']==i]['PM2.5'])[int(n)-1] for n in months] for i in list_district]

fig = go.Figure()
for pm,d,c in zip(list_PM, list_district, pal):
    fig.add_trace(go.Scatterpolar(r = pm, theta=months, fill= None,
                                  name=str(d), marker = dict(color = c)))

fig.update_layout(polar = dict(radialaxis = dict(visible = True, range=[0, 70]),
                               angularaxis = dict(rotation=90)),
                  showlegend=True, width=720, height=720,
                  font = dict(size=14))

fig.show()

#### 6. Radial Plot
- Radial Plot은 직교 좌표 대신 극좌표를 사용하는 막대 차트를 기반으로 합니다. - 이 차트 유형은 서로 멀리 떨어져 있는 범주를 비교할 때 불편하지만 주의를 끌기에 탁월한 선택입니다. 인포그래픽에서 사용할 수 있습니다.
- 아래 예는 2019년 1월 25개 지역의 평균 PM2.5를 보여주는 방사형 플롯의 예를 보여줍니다.

In [ ]:
#set color palette, lower and max values 
pal = list(sns.color_palette(palette='CMRmap_r', n_colors=len(list_district)).as_hex())
lowerLimit = 0
max_v = df_19['PM2.5'].max()

def radial_plot(input_df, column_name, title):
    input_df.reset_index(inplace=True, drop=True)
    plt.figure(figsize=(12,12))
    ax = plt.subplot(111, polar=True)
    plt.axis()
    
    heights = input_df[column_name]
    width = 2*np.pi / len(input_df.index)

    indexes = list(range(1, len(input_df.index)+1))
    angles = [element * width for element in indexes]

    bars = ax.bar(x=angles, height=heights, width=width, bottom=lowerLimit,
                  linewidth=1, edgecolor="white", color=pal)
    
    labelPadding = 2

    for bar, angle, height, label in zip(bars, angles, heights, list_district):
        rotation = np.rad2deg(angle)
        alignment = ""
      
        if angle >= np.pi/2 and angle < 3*np.pi/2:
            alignment = "right"
            rotation = rotation + 180
        else: 
            alignment = "left"

        ax.text(x=angle, y=lowerLimit + bar.get_height() + labelPadding,
                s=label, ha=alignment, va='center', rotation=rotation, 
                rotation_mode="anchor")
    
        ax.set_thetagrids([], labels=[])
        plt.title("Average PM2.5 // " + title)
    return ax

In [ ]:
keep_sname = []
order = range(len(listdf_monthly19))

for i in order:
    radial_plot(listdf_monthly19[i], 'PM2.5', list_YM19[i])
    keep_sname.append('rad_bar_' + str(i) + '.png')
    plt.savefig('rad_bar_' + str(i) + '.png')
    plt.show()

#### 8. Ridge plot : 중첩 밀도 표시
- 릿지 플롯은 축을 타임라인으로 설정하여 여러 시계열 데이터와 함께 사용할 수 있습니다.
- 다음 예는 2019년 한 지역의 PM2.5 밀도가 있는 Ridge 플롯의 예를 보여줍니다.

In [ ]:
#change color scale for 12 months
pal = list(sns.color_palette(palette='viridis', n_colors=len(list_month19)).as_hex())

def kde_ridge(df_input, col_name, time, title):
    sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

    # Initialize the FacetGrid object
    g = sns.FacetGrid(df_input, row= time, hue=time, aspect=15, height=0.65, palette=pal)

    # Draw the densities in a few steps
    g.map(sns.kdeplot, col_name,
          bw_adjust=.5, clip_on=False,fill=True, alpha=1, linewidth=1.5)

    g.map(sns.kdeplot, col_name, clip_on=False, color="w", lw=2, bw_adjust=.5)

    # passing color=None to refline() uses the hue mapping
    g.map(plt.axhline, y=0, linewidth=2, linestyle="-", color=None, clip_on=False)
    
    def label(x, color, label):
        ax = plt.gca()
        ax.text(0, .2, label, fontweight="bold", color=color,ha="left", va="center", transform=ax.transAxes)
    
    g.map(label, time)

    # Set the subplots to overlap
    g.fig.subplots_adjust(hspace=-.25)

    # Remove axes details that don't play well with overlap
    g.set_titles("")
    g.set(yticks=[], ylabel="", xlabel= title + ' PM2.5')
    g.despine(bottom=True, left=True)

    return g

In [ ]:
#use hourly data
df_hour19 = df[df['Year']=='2019'] 
df_hd19 = [df_hour19[df_hour19['District']==i] for i in list_district]

keep_sname = []
order = range(len(list_district))
for i in order:
    kde_ridge(df_hd19[i], 'PM2.5', 'YM', list_district[i])
    keep_sname.append('kde_' + str(i) + '.png')
    plt.savefig('kde_' + str(i) + '.png')
    plt.show()